In [ ]:
# %pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
%reset -f
import plotly.express as px
from graphviz import Digraph
from collections import defaultdict
from itertools import product
import numpy as np

In [16]:
# Параметры нагрузки
LAM = 1
b = 2
P = [0.6, 0.25, 0.15]
SERVED_SIZE=6
NUM_SERVERS = 3
# QUEUE_SIZES=[3] 

# redgry
#LAM = 0.5; b = 10; P = [0.5, 0.5]; QUEUE_SIZES = [2,1]

# Print
MU = 1/b
assert sum(P) == 1
print(f"λ = {LAM}", f"{b=}", f"μ = {MU}", sep="\n")
for i in range(len(P)):
  print(f'p{i+1} = {P[i]}')

λ = 1
b = 2
μ = 0.5
p1 = 0.6
p2 = 0.25
p3 = 0.15


In [18]:
states = {
    state: {}
    for state in product(*[range(SERVED_SIZE + 1) for _ in range(NUM_SERVERS)])
}
for in_state in states:
    for idx in range(NUM_SERVERS):
        out_state = list(in_state)
        out_state[idx] += 1
        
        if sum(out_state) > SERVED_SIZE:
            continue
        
        out_state = tuple(out_state)
        states[in_state][out_state] = P[idx] * LAM

    for idx in range(NUM_SERVERS):
        if in_state[idx] > 0:
            out_state = list(in_state)
            out_state[idx] -= 1
            
            out_state = tuple(out_state)
            states[in_state][out_state] = MU



In [20]:
for state, transitions in states.items():
    print(f"State: {state}")
    for next_state, prob in transitions.items():
        print(f"  -> Next State: {next_state} with Probability: {prob:.4f}")

State: (0, 0, 0)
  -> Next State: (1, 0, 0) with Probability: 0.6000
  -> Next State: (0, 1, 0) with Probability: 0.2500
  -> Next State: (0, 0, 1) with Probability: 0.1500
State: (0, 0, 1)
  -> Next State: (1, 0, 1) with Probability: 0.6000
  -> Next State: (0, 1, 1) with Probability: 0.2500
  -> Next State: (0, 0, 2) with Probability: 0.1500
  -> Next State: (0, 0, 0) with Probability: 0.5000
State: (0, 0, 2)
  -> Next State: (1, 0, 2) with Probability: 0.6000
  -> Next State: (0, 1, 2) with Probability: 0.2500
  -> Next State: (0, 0, 3) with Probability: 0.1500
  -> Next State: (0, 0, 1) with Probability: 0.5000
State: (0, 0, 3)
  -> Next State: (1, 0, 3) with Probability: 0.6000
  -> Next State: (0, 1, 3) with Probability: 0.2500
  -> Next State: (0, 0, 4) with Probability: 0.1500
  -> Next State: (0, 0, 2) with Probability: 0.5000
State: (0, 0, 4)
  -> Next State: (1, 0, 4) with Probability: 0.6000
  -> Next State: (0, 1, 4) with Probability: 0.2500
  -> Next State: (0, 0, 5) with

In [19]:
def make_state_graph(states):
  g = Digraph(engine='fdp')
  g.attr(overlap='scalexy')
  for in_state in states.keys():
    id = str(in_state)
    g.node(id, id)
  for in_state, out_states in states.items():
    for out_state, prob in out_states.items():
      in_id = str(in_state)
      out_id = str(out_state)
      assert sum(in_state) != sum(out_state)
      color = 'green' if sum(in_state) < sum(out_state) else 'red'
      g.edge(in_id, out_id, f'{prob:.3}', color=color, fontcolor=color)
  return g

display(make_state_graph(states))

In [7]:
def make_state_indices(states):
  idx = {}
  for i, state in enumerate(states):
    idx[state] = i
    idx[i] = state
  return idx

def make_transition_rate_matrix(states, state_idx):
  N = len(states)
  m = np.zeros((N, N))
  for in_state, out_states in states.items():
    in_idx = state_idx[in_state]
    for out_state, rate in out_states.items():
      out_idx = state_idx[out_state]
      m[in_idx, out_idx] = rate
  for i in range(N):
    m[i, i] = -sum(m[i, :])
  return m

state_idx = make_state_indices(states)
transition_rate_matrix = make_transition_rate_matrix(states, state_idx)

fig = px.imshow(
  transition_rate_matrix,
  text_auto = True,
  height = 800,
  color_continuous_scale = [[0, 'red'], [0.5, '#f8f8f8'], [1, 'green']],
  color_continuous_midpoint = 0.0,
)
fig.show()

In [8]:
def solve_states(transition_rate_matrix):
  shape = transition_rate_matrix.shape
  assert len(shape) == 2
  assert shape[0] == shape[1]
  N = shape[0]
  equations = transition_rate_matrix.copy().transpose()
  equations[0, :] = np.ones(N)
  bvector = np.zeros(N)
  bvector[0] = 1
  return np.linalg.solve(equations, bvector)

probs = solve_states(transition_rate_matrix)

for i in range(len(states)):
  print(f'{i}\t{state_idx[i]}\t{probs[i]:.4}')

0	(0, 0, 0)	0.0124
1	(0, 0, 1)	0.002481
2	(0, 1, 0)	0.0124
3	(0, 1, 1)	0.002481
4	(1, 0, 0)	0.03473
5	(1, 0, 1)	0.006946
6	(1, 1, 0)	0.03473
7	(1, 1, 1)	0.006946
8	(2, 0, 0)	0.09725
9	(2, 0, 1)	0.01945
10	(2, 1, 0)	0.09725
11	(2, 1, 1)	0.01945
12	(3, 0, 0)	0.2723
13	(3, 0, 1)	0.05446
14	(3, 1, 0)	0.2723
15	(3, 1, 1)	0.05446


In [9]:
y = np.array(P) * (LAM / MU) # Нагрузка
pbusy = np.zeros(len(P)) # Загрузка
L = np.zeros(len(P)) # Длина очереди
m = np.zeros(len(P)) # Число заявок
prod = np.zeros(len(P)) # Производительность
loss = np.zeros(len(P)) # Вероятность потери
wtime = np.zeros(len(P)) # Время ожидания
ttime = np.zeros(len(P)) # Время пребывания

for si, prob in enumerate(probs):
  state = state_idx[si]
  for did, work in enumerate(state):
    assert work >= 0 and work <= QUEUE_SIZES[did] + 1
    if work > 0:
      pbusy[did] += prob
    if work > 1:
      L[did] += prob * (work-1)
    m[did] += prob * work
    if work == QUEUE_SIZES[did] + 1:
      loss[did] += prob
prod = LAM * np.array(P) * (1 - loss)
wtime = L / prod
ttime = m / prod

print(f'Нагрузка      {y = } Σ {sum(y)}')
print(f'Загрузка      {pbusy = } Σ {sum(pbusy)}')
print(f'Длина очереди {L}, Σ {sum(L)}')
print(f'Число заявок  {m}, Σ {sum(m)}')
print(f'Время ожидания       {wtime}, Σ {sum(L) / sum(prod)}')
print(f'Время пребывания     {ttime}, Σ {sum(m) / sum(prod)}')
print(f'Вероятность потери   {loss}, Σ {sum(loss * P)}')
print(f'Производительность   {prod}, Σ {sum(prod)}')

Нагрузка      y = array([2.8, 1. , 0.2]) Σ 4.0
Загрузка      pbusy = array([0.97023101, 0.5       , 0.16666667]) Σ 1.6368976740493768
Длина очереди [1.54036675 0.         0.        ], Σ 1.5403667539890447
Число заявок  [2.51059776 0.5        0.16666667], Σ 3.1772644280384212
Время ожидания       [63.50515464  0.          0.        ], Σ 37.641125121241494
Время пребывания     [103.50515464  40.          40.        ], Σ 77.64112512124147
Вероятность потери   [0.65348893 0.5        0.16666667], Σ 0.5907755814876555
Производительность   [0.02425578 0.0125     0.00416667], Σ 0.040922441851234434


In [10]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
